In [3]:
import librosa
import numpy as np

import os
import time

import boto3
from botocore.exceptions import ClientError

import gluonnlp as nlp

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from transformers import AutoTokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
from kobert.utils import get_tokenizer

from torchvision.models.resnet import ResNet, BasicBlock

In [4]:
import pandas as pd

In [5]:
# BERT 모델, Vocabulary 불러오기
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size=768,
                 num_classes=6,  ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size, num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids=token_ids, token_type_ids=segment_ids.long(),
                              attention_mask=attention_mask.float().to(token_ids.device), return_dict=False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)


In [6]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, vocab=vocab, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i],))

    def __len__(self):
        return (len(self.labels))


In [7]:
class ResNetModel(ResNet):
    def __init__(self, num_classes=6):
        super(ResNetModel, self).__init__(BasicBlock, [2, 2, 2, 2], num_classes=num_classes)
        self.conv1 = nn.Conv2d(4, 64, kernel_size=7, stride=2, padding=3, bias=False)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

In [8]:
device = torch.device('cpu')

In [9]:
# Setting prediction parameters
max_len = 60
batch_size = 64
learning_rate = 5e-5

print("Loading BERT model...")
# Load pre-trained model (weights)
bertmodel, vocab = get_pytorch_kobert_model()

# Load tokenizer from a local directory
# kobert_tokenizer = AutoTokenizer.from_pretrained("kobert_tokenizer", use_fast=False)
# tok = kobert_tokenizer.tokenize
print("Loading BERT tokenizer...")
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

PATH = '/Users/yoohajun/Library/Mobile Documents/com~apple~CloudDocs/Hajun/Graduate_project/fastapi/KoBERT/'
kobert_model = BERTClassifier(bertmodel, dr_rate=0.5)
kobert_model.load_state_dict(torch.load(PATH + 'model_state_dict.pt', map_location=device))

Loading BERT model...
using cached model. /Users/yoohajun/Library/Mobile Documents/com~apple~CloudDocs/Hajun/Graduate_project/fastapi/Diffusion/.cache/kobert_v1.zip
using cached model. /Users/yoohajun/Library/Mobile Documents/com~apple~CloudDocs/Hajun/Graduate_project/fastapi/Diffusion/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
Loading BERT tokenizer...
using cached model. /Users/yoohajun/Library/Mobile Documents/com~apple~CloudDocs/Hajun/Graduate_project/fastapi/Diffusion/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


<All keys matched successfully>

In [10]:
# Define the path to the saved model
model_path = '/Users/yoohajun/Library/Mobile Documents/com~apple~CloudDocs/Hajun/Graduate_project/fastapi/resnet-model/pytorch_resnet.pt'
# Load the saved model
checkpoint = torch.load(model_path, map_location=torch.device('cpu'))
resnet_model = ResNetModel(num_classes=6)
resnet_model.load_state_dict(checkpoint['model_state_dict'])


<All keys matched successfully>

In [11]:
def scale_to_range(arr, target_range=(0, 1)):
    # Calculate exponential values for each element
    exp_arr = np.exp(arr)

    # Calculate the sum of all exponential values
    exp_sum = np.sum(exp_arr)

    # Calculate probability for each element by dividing its exponential value by the sum
    probs = exp_arr / exp_sum

    # Scale probabilities to target range
    scaled = (probs * (target_range[1] - target_range[0])) + target_range[0]

    return scaled

In [12]:
def softmax(vals, idx):
    valscpu = vals.cpu().detach().squeeze(0)
    a = 0
    for i in valscpu:
        a += np.exp(i)
    return ((np.exp(valscpu[idx])) / a).item() * 100


In [13]:
def text_predict(predict_sentence, model=kobert_model):
    # print("predictsentence start:", predict_sentence)
    start = time.time()
    text_label = ['regular', 'help', 'robbery', 'sexual', 'theft', 'violence']
    data = [predict_sentence]
    # dataset_another = [data]

    transform = nlp.data.BERTSentenceTransform(tok, max_len, pad=True, pair=False)
    tokenized = transform(data)
    model.eval()

    # print([tokenized[0]])
    # token_ids = torch.tensor([tokenized[0]]).to(device)
    # segment_ids = torch.tensor([tokenized[2]]).to(device)
    token_ids = torch.tensor(np.array([tokenized[0]])).to(device)
    valid_length = [tokenized[1]]
    segment_ids = torch.tensor(np.array([tokenized[2]])).to(device)

    result = model(token_ids, valid_length, segment_ids)
    # print(result)
    idx = result.argmax().cpu().item()
    out_prob = result.detach().cpu().numpy()[0]
    # print(out_prob)
    # print("대사의 카테고리는:", text_label[idx])
    # print("대사 신뢰도는:", "{:.2f}%".format(softmax(result, idx)))
    end = time.time() - start
    # print("text predict 걸린 시간:", end)
    return out_prob

In [14]:
def audio_predict(file_location, model=resnet_model):
    try:
        audio_data, sr = librosa.load(file_location, sr=44100, duration=5)

        # Define label names
        label_names = ['regular', 'help', 'robbery', 'sexual', 'theft', 'violence']

        # Calculate the spectrogram of the audio data
        spec = librosa.feature.melspectrogram(y=audio_data, sr=sr)

        # Convert the spectrogram to decibels
        spec_db = librosa.power_to_db(spec, ref=np.max)

        # Add an additional channel to the spectrogram
        spec_db = np.repeat(spec_db[:, :, np.newaxis], 4, axis=2)

        # Resize the spectrogram to match the input shape of the model
        spec_resized = np.resize(spec_db, (1, 4, 128, 128))

        # Normalize the spectrogram by z-score
        mean = np.mean(spec_resized)
        std = np.std(spec_resized)
        spec_resized = (spec_resized - mean) / std

        # Convert the spectrogram to a tensor and move it to the device
        spectrogram_tensor = torch.tensor(spec_resized, dtype=torch.float).to(device)

        # Set the model to evaluation mode
        model.eval()

        # Predict the probabilities for each class
        with torch.no_grad():
            out = model(spectrogram_tensor)

        # Get the index of the class with the highest probability
        predicted_class_index = torch.argmax(out, dim=1)

        label_index = predicted_class_index.item()

        # print("음성의 카테고리는:", label_names[label_index])
        # print("음성 신뢰도는:", "{:.2f}%".format(softmax(out, label_index)))

        return out.detach().cpu().numpy()[0]

    except Exception as e:
        print(f'Error: {e}')
        return None

In [15]:
df = pd.read_csv('./audio_text_diffusion_train.csv', encoding='utf8')

In [16]:
df

,Unnamed: 0,audio_id,audio_dir,note,category_02
0,0,15.실내_906628_label,/Users/yoohajun/Desktop/grad_audio/diffusion/t...,NaN,실내
1,1,15.실내_903897_label,/Users/yoohajun/Desktop/grad_audio/diffusion/t...,NaN,실내
2,2,15.실내_918134_label,/Users/yoohajun/Desktop/grad_audio/diffusion/t...,NaN,실내
3,3,15.실내_654528_label,/Users/yoohajun/Desktop/grad_audio/diffusion/t...,NaN,실내
4,4,15.실내_661859_label,/Users/yoohajun/Desktop/grad_audio/diffusion/t...,NaN,실내
...,...,...,...,...,...
402,402,3.절도범죄_1240207_label,/Users/yoohajun/Desktop/grad_audio/diffusion/t...,소매치기 잡아라,절도범죄
403,403,3.절도범죄_1240211_label,/Users/yoohajun/Desktop/grad_audio/diffusion/t...,소매치기 잡아라,절도범죄
404,404,3.절도범죄_1240983_label,/Users/yoohajun/Desktop/grad_audio/diffusion/t...,저 놈 잡아라,절도범죄
405,405,3.절도범죄_1240989_label,/Users/yoohajun/Desktop/grad_audio/diffusion/t...,저 놈 잡아라,절도범죄


In [17]:
# 음성 파일 id, 음성 파일 경로, note, 음성 파일 븐류 -> 1차 csv

# 오디오 확률 리스트, 텍스트 확률 리스트, 라벨

# 리스트 nested 해제 , 라벨

# diffusion layer ml 학습 - decision tree

In [18]:
# define a function to apply to each row of the DataFrame
def add_prediction(row):
    # call the audio_predict function with the audio_dir value
    audio_prediction = audio_predict(row['audio_dir'])
    # add the prediction values as a new column to the DataFrame
    if audio_prediction is not None:
        # add the prediction values as a new column to the DataFrame
        row['audio_prediction'] = audio_prediction.tolist()
    else:
        row['audio_prediction'] = None

    text_prediction = text_predict(row['note'])

    if text_prediction is not None:
        # add the prediction values as a new column to the DataFrame
        row['text_prediction'] = text_prediction.tolist()
    else:
        row['text_prediction'] = None

    return row


In [19]:
df

,Unnamed: 0,audio_id,audio_dir,note,category_02
0,0,15.실내_906628_label,/Users/yoohajun/Desktop/grad_audio/diffusion/t...,NaN,실내
1,1,15.실내_903897_label,/Users/yoohajun/Desktop/grad_audio/diffusion/t...,NaN,실내
2,2,15.실내_918134_label,/Users/yoohajun/Desktop/grad_audio/diffusion/t...,NaN,실내
3,3,15.실내_654528_label,/Users/yoohajun/Desktop/grad_audio/diffusion/t...,NaN,실내
4,4,15.실내_661859_label,/Users/yoohajun/Desktop/grad_audio/diffusion/t...,NaN,실내
...,...,...,...,...,...
402,402,3.절도범죄_1240207_label,/Users/yoohajun/Desktop/grad_audio/diffusion/t...,소매치기 잡아라,절도범죄
403,403,3.절도범죄_1240211_label,/Users/yoohajun/Desktop/grad_audio/diffusion/t...,소매치기 잡아라,절도범죄
404,404,3.절도범죄_1240983_label,/Users/yoohajun/Desktop/grad_audio/diffusion/t...,저 놈 잡아라,절도범죄
405,405,3.절도범죄_1240989_label,/Users/yoohajun/Desktop/grad_audio/diffusion/t...,저 놈 잡아라,절도범죄


In [20]:
# fill NaN values with empty string
df = df.fillna('empty')

In [21]:
from tqdm import tqdm
tqdm.pandas()
# apply the add_prediction function to each row of the DataFrame
df = df.progress_apply(add_prediction, axis=1)

  1%|          | 5/407 [00:02<03:19,  2.02it/s]/var/folders/3p/vhd92h0x7cb73w5m2xzsn2jr0000gn/T/ipykernel_18169/1743214241.py:23: RuntimeWarning: invalid value encountered in divide
  spec_resized = (spec_resized - mean) / std
  5%|▍         | 19/407 [00:08<02:23,  2.70it/s]/var/folders/3p/vhd92h0x7cb73w5m2xzsn2jr0000gn/T/ipykernel_18169/1743214241.py:3: UserWarning: PySoundFile failed. Trying audioread instead.
  audio_data, sr = librosa.load(file_location, sr=44100, duration=5)
/opt/anaconda3/envs/pytorch-grad/lib/python3.8/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Error: [Errno 2] No such file or directory: '/Users/yoohajun/Desktop/grad_audio/diffusion/train/15.실내_688645_label.wav'


100%|██████████| 407/407 [02:32<00:00,  2.67it/s]


In [22]:
# print the resulting DataFrame with the new 'prediction' column
df.head()

,Unnamed: 0,audio_id,audio_dir,note,category_02,audio_prediction,text_prediction
0,0,15.실내_906628_label,/Users/yoohajun/Desktop/grad_audio/diffusion/t...,empty,실내,"[-11.508453369140625, -8.252114295959473, -23....","[1.985169768333435, -1.2518784999847412, 1.072..."
1,1,15.실내_903897_label,/Users/yoohajun/Desktop/grad_audio/diffusion/t...,empty,실내,"[17.464433670043945, -21.854764938354492, -7.6...","[1.985169768333435, -1.2518784999847412, 1.072..."
2,2,15.실내_918134_label,/Users/yoohajun/Desktop/grad_audio/diffusion/t...,empty,실내,"[3.1379899978637695, 1.7926372289657593, -12.4...","[1.985169768333435, -1.2518784999847412, 1.072..."
3,3,15.실내_654528_label,/Users/yoohajun/Desktop/grad_audio/diffusion/t...,empty,실내,"[28.44495964050293, -16.67407989501953, -4.198...","[1.985169768333435, -1.2518784999847412, 1.072..."
4,4,15.실내_661859_label,/Users/yoohajun/Desktop/grad_audio/diffusion/t...,empty,실내,"[nan, nan, nan, nan, nan, nan]","[1.985169768333435, -1.2518784999847412, 1.072..."


In [23]:
df.to_csv('./audio_text_label.csv')